# Разработка модели классификации комментариев

Сформулируем цель проекта, определимся с задачей и приступим к анализу данных.

### Цель проекта:
Разработать модель машинного обучения, классифицирующую комментарии на "токсичные" и "позитивные".

Метрика - f1-score.

Перед нами задача классификации (не регрессии).

### Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import re
import time

import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

# 0. Предобработка и анализ данных

Посмотрим на имеющиеся в нашем распоряжении данные.

In [2]:
comment_data = pd.read_csv('/datasets/toxic_comments.csv')
comment_data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [3]:
comment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Оу. Перед нами (мной) ИНОСТРАННЫЙ (для меня) язык. Интересно. 

Почти 160 тысяч размеченных комментариев. NAN-пропусков нет. Сходу не видно "токсичных комментариев". Оттого и не ясно, например, каково соотношение классов и какой, при необходимости, стоит делать "перевес" в пользу одного из классов. Посмотрим на это.

Из "не нужного" вижу знаки переноса строки (это которые \n) + некий набор двоеточий (::::). Что он может значит - не знаю, однако, сомневаюсь, что мне вообще нужно об этом узнавать. Чистить, скорее всего, будем предметно - удаляя описанные артефакты (конечно не в ручную). 

Но сперва - баланс классов.

In [4]:
comment_data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

90 на 10. А я всегда говорил - "Люди - хорошие!"

Стало быть дисбаланс классов налицо - следовательно, будем готовы применить МЕТОДЫ. Какие именно я сейчас не помню, но когда ПРИДЁТ ВРЕМЯ, я всё резко вспомню и "распишу в лучшем виде" (да, сегодня в комментариях к проекту - много комментариев к комментариям, смиритесь. И в очередной раз - СПАСИБО за то, что вы делаете. Вы очень крутые и занимаетесь важным делом - учите людей).

Вспоминая работу с категориальными признаками, "опустим" все буквы в нижний регистр.

In [5]:
comment_data['text'] = comment_data['text'].str.lower()

Посмотрим несколько случайных комментариев из "тела" датасета + "хвост", что бы понять, с чем нам ещё (теоретически) предстоит работать. 

In [6]:
comment_data.loc[150, 'text']

'"\n\n socialistm? \n\nthere are two important features of smith\'s concept of the ""invisible hand"". first, smith was not advocating a social policy (that people should act in their own self interest), but rather was describing an observed economic reality (that people do act in their own interest). second, smith was not claiming that all self-interest has beneficial effects on the community. he did not argue that self-interest is always good; he merely argued against the view that self-interest is necessarily bad. it is worth noting that, upon his death, smith left much of his personal wealth to charity.\n\ngood!  let\'s all make sure we put forth the idea that adam smith was a socialist.  that\'s the wikipedia way!"'

Следующий "косяк" (косяк ли это?) - наличие в теле комментариев "особенной" разметки для апострофов- вместе с обраным слешем. Думаю здесь нам потребуется замена \' на "чистый апостроф". С другой стороны, ведь под каждым апострофом в английском языке (ну почти под каждым) скрывается слово - глаголы типа is, are, am и тому подобное. Следовательно, мы, теоретически, можем "раскрывать" апострофы в виде дополнительного слова. Да, звучит СЛОЖНО (только потому, что я не знаю методов, которыми это можно сделать БЫСТРО) и НЕОЧЕВИДНО. Но не будем отметать такой вариант.

Следующий!

In [7]:
comment_data.loc[1500, 'text']

'there is no evidence that this block has anything to do with sock accounts. my use of my accounts was completely legitimate. and contrary to what rodhullandemu is trying to imply, the only accounts that i used in the police talk page are mr3003nights and this one (which is a completely legitimate practice). i was blocked for a discussion in the talk page of the article on the policenot for editing the actual article. please read that discussion i linked to and see it for yourself that it was pure unjustified retaliation for offering evidence rodhullandemu could not answer http://en.wikipedia.org/wiki/talk:police#why_reiterate_.22reduce_civil_disorder.22_instead_of_protection_of_property.3f 69.228.251.134'

В наличии ссылка на википедию + IP адрес. Зачем они здесь? Автор, наверняка, так выстраивал "линию защиты" или обвинения. Нам же это, скорее всего, не поможет, т.к. ссылка будет воспринята как одно ОГРОМНОЕ слово и, скорее, помешает обучению модели, чем поможет. Хотя это не точно. И тем не менее, БУДЕМ УДАЛЯТЬ. Ссылки, как правило, начинаются на http(s) и идут НЕПРЕРЫВНО. Следовательно, мы можем РАСЧИСТИТЬ датасет удаляя все ДЛИННЫЕ слова, начинающиеся с http. 

Что делать с ip пока не понятно. Высок риск потерять комментарии с ценами и прочим. Ведь у нас магазин. И писать цены и сравнение в комменты - вполне обыденная практика. С другой стороны, наличие цифр в модели, врядли поможет отсортировать комментарии на положительные и токсичные. Ведь никого не обижает цифра 8. Или кого-то обижает? Так или иначе, если из предложения "У тебя мозги как у 8-летнего школьника" убрать цифру 8, смысл сохранится. Следовательно, цифры нам, скорее не нужны, чем нужны. Уберем.

Next!

In [8]:
comment_data.loc[15000, 'text']

'ingrid tamuyeye is also a young athlete but does not wish her name to be published on the interner and daniella ashaju is one of the best spokes people of youngsters but does not want her name to be published.'

Всё в порядке. Всем бы так.

In [9]:
comment_data.loc[150000, 'text']

'"\nthe addition was a ""wikipedia situation where a fair use image shouldn\'t be used"", not a ""counterexample showing what fair-use is not"", as the section is. "'

Вроде бы тоже всё норм, только мне не нравится куча кавычек. Да, я сам ОБОЖАЮ ими пользоваться - чтобы уточнить смысл. Но, полагаю, модели ML на это будет плевать. По крайней мере в том виде, в каком я её себе сейчас представляю. Стало быть кавычки - тоже под нож.

Итак, перечень "артефактов" для предобработки данных:
- \n
- :::: (думаю, тут можно удалить все двоеточия по тексту. Особо смысл от этого не потеряется, мне кажется)
- \' -> ' (замена каким-то образом апострофо-слешей на апострофы).
- http... (ссылки)
- Цифры
- """"""" (вагоны кавычек. Убираем всё)

Для модели того типа, который я сейчас себе представляю, данные не должны содержать ничего, кроме слов. Ни знаков препинания (да, можно и с ними, но я сейчас не особо понимаю, как сохранить нужные з.п. и удалить не нужные з.п.), ни цифр, ни ссылок, ни апострофов и т.д.

>UPD. Тут ещё подумал, что так же, в принципе, смысла не имеют никнеймы пользователей в комментариях. Но как их всех отследить, я пока не представляю.

Поехали!

# 1. Подготовка

Что нам нужно оставить в тексте? В принципе, только буквы. Однако, сначала нам нужно удалить ссылки http, что бы в строках, после удаления НЕ-букв, не остались разорванные на куски ссылки, которые будут совершенно бесполезны и даже вредны при обучении модели и подготовке данных. 

Воспользуемся re.

> Разобраться в том, как работают регулярки было не особо сложно. Пользовался https://regex101.com/

In [10]:
%%time 

def re_sub_http(string):
    pattern_http = 'http.*\s'
    return re.sub(pattern_http, ' ', string)

comment_data['text'] = comment_data['text'].apply(re_sub_http)

CPU times: user 365 ms, sys: 262 µs, total: 365 ms
Wall time: 404 ms


Теперь убираем символ переноса на другую строку. 

Я не разобрался, как удалить этот символ из строк с помощью регулярок - вероятно потому, что \n является встроенным оператором библиотеки. Поэтому воспользуюсь встроенным "реплейсом".

In [11]:
comment_data['text'] = comment_data['text'].replace('/n', ' ')
comment_data.loc[150, 'text']

'"\n\n socialistm? \n\nthere are two important features of smith\'s concept of the ""invisible hand"". first, smith was not advocating a social policy (that people should act in their own self interest), but rather was describing an observed economic reality (that people do act in their own interest). second, smith was not claiming that all self-interest has beneficial effects on the community. he did not argue that self-interest is always good; he merely argued against the view that self-interest is necessarily bad. it is worth noting that, upon his death, smith left much of his personal wealth to charity.\n\ngood!  let\'s all make sure we put forth the idea that adam smith was a socialist.  that\'s the wikipedia way!"'

И, наконец, очистим данные от всего, кроме букв. Маленьких - большие мы все уже "съели".

In [12]:
def re_sub_letters(string):
    return re.sub(r'[^a-z ]', ' ', string)

comment_data['text'] = comment_data['text'].apply(re_sub_letters)             

In [13]:
comment_data.loc[1500, 'text']

'there is no evidence that this block has anything to do with sock accounts  my use of my accounts was completely legitimate  and contrary to what rodhullandemu is trying to imply  the only accounts that i used in the police talk page are mr    nights and this one  which is a completely legitimate practice   i was blocked for a discussion in the talk page of the article on the policenot for editing the actual article  please read that discussion i linked to and see it for yourself that it was pure unjustified retaliation for offering evidence rodhullandemu could not answer                '

Да, теперь перед нами есть текст с кучей пробелов, однако, в процессе разделения всё встанет на свои места и ненужные пробелы будут "ликвидированы". Да, мы можем их устранить несколько раз прогнав замену "  " на " " (двойной пробел на пробел), однако, зачем? 

Дальнейшее движение возможно в 2 направлениях - в сторону стемминга и лемматизации. Логически, лемматизация "лучше", т.к. позволяет почти наверняка получить более "точные" слова, а значит меньшее число и большее качество признаков для обучения модели. Тем не менее, стемминг работает быстрее и, в нашем случае, почти наверняка только он сможет справить с задачей в "приемлемые" сроки. 

Попробуем оба варианта и посмотрим на длительность и результат.

Сначала стемминг. А ещё более сначала - токенизация.

In [14]:
%%time
def tokenizer(string):
    return nltk.word_tokenize(string)
    
comment_tokenized = comment_data['text'].apply(tokenizer)

CPU times: user 1min 6s, sys: 604 ms, total: 1min 6s
Wall time: 1min 7s


In [15]:
comment_tokenized[0]

['explanation',
 'why',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'hardcore',
 'metallica',
 'fan',
 'were',
 'reverted',
 'they',
 'weren',
 't',
 'vandalisms',
 'just',
 'closure',
 'on',
 'some',
 'gas',
 'after',
 'i',
 'voted',
 'at',
 'new',
 'york',
 'dolls',
 'fac',
 'and',
 'please',
 'don',
 't',
 'remove',
 'the',
 'template',
 'from',
 'the',
 'talk',
 'page',
 'since',
 'i',
 'm',
 'retired',
 'now']

Да, среди "разделенных слов" явно видно наличие "следов" кривой предобработки - отдельностоящие буквы s, t, n и т.п. от разделенных don't, i'm и т.д. Да, косяк, Однако, полагаю, что для нашей задачи это не будет большой потерей, т.к. первоформа слов (насколько мне известно), не включает отрицание. Хотя, вероятно, я не прав. Посмотрим на результат стемминга и подумаем о перспективах. 

С другой стороны, полагаю, что слова-в-одну-букву не окажут особого влияния на смысл предложения. С третьей стороны, их существование максимум добавит 28 признаков, что в масштабе всего проекта - капля в море. 

In [16]:
%%time
stemmer = PorterStemmer()
test_data = comment_tokenized[2]
stemmer_output = ' '.join([stemmer.stem(w) for w in test_data])

CPU times: user 1.25 ms, sys: 0 ns, total: 1.25 ms
Wall time: 1.26 ms


In [17]:
%%time 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in test_data])

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


CPU times: user 2.71 s, sys: 96 ms, total: 2.81 s
Wall time: 3.22 s


Первый, очень важный пункт - на представленных данных Лемматайзер БЫСТРЕЕ стемера. Это хорошо. Сравним конечные результаты с исходным текстом по критерию "адекватности" перевода. Да, это чисто субъективно, однако, думаю, в этом смысле мы все будем близки к некоторому общему виду "как должно быть".

>UPD. Спустя 2 запуска, стеммер "стеммит" быстрее. Хммм.

In [18]:
print(test_data)
print()
print(stemmer_output)
print()
print(lemmatized_output)

['hey', 'man', 'i', 'm', 'really', 'not', 'trying', 'to', 'edit', 'war', 'it', 's', 'just', 'that', 'this', 'guy', 'is', 'constantly', 'removing', 'relevant', 'information', 'and', 'talking', 'to', 'me', 'through', 'edits', 'instead', 'of', 'my', 'talk', 'page', 'he', 'seems', 'to', 'care', 'more', 'about', 'the', 'formatting', 'than', 'the', 'actual', 'info']

hey man i m realli not tri to edit war it s just that thi guy is constantli remov relev inform and talk to me through edit instead of my talk page he seem to care more about the format than the actual info

hey man i m really not trying to edit war it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info


Лемматайзер также показал себя лучше с точки зрения корректного извлечения слов. Да, не везде, однако "косяки" стеммера несравнимо больше, чем в случае лемм. Напишем функцию для лемматизации всего массива данных и оценим сроки выполнения этой части предобработки.

In [19]:
lemmatizer = WordNetLemmatizer()

def data_lemmas(text):
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in text])
    return lemmatized_output

In [20]:
%%time
comment_lemmas = comment_tokenized.apply(data_lemmas)

CPU times: user 55.3 s, sys: 228 ms, total: 55.5 s
Wall time: 55.7 s


Минута на лемматизацию - супер!

Воспользуемся значениями TF-IDF в качестве признков для модели.

In [21]:
%%time
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
comment_tf_idf = count_tf_idf.fit_transform(comment_lemmas)
comment_tf_idf.shape

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


CPU times: user 10.6 s, sys: 196 ms, total: 10.8 s
Wall time: 10.9 s


(159571, 153504)

160 тысяч на 160 тысяч значений. Причем не нулей и единиц, а флоатов - ведь tf-idf почти никогда не равен единице. 

Предположим, что датасет у нас готов. Нарежем его на обучающую, валидационную и тестовую части (60 к 20 к 20) и подберем несколько моделей для обучения и прогнозирования. Метрика по-прежнему F1.

Такое распределение выбрал, т.к. планирую использовать гридсёрч и нужны будут обучающие + валидационные данные.

>Столкнулся сам + читал в слаке, что платформа практикума с большим скрипом тянула рассчеты по этому проекту. Не знаю, ребят, что вы сделали, но сейчас будто лечу в самолёте первым классом) Всё считается быстро) Спасибо вам!

In [22]:
features = comment_tf_idf
target = comment_data['toxic']

f_train_valid, f_test, t_train_valid, t_test = train_test_split(features, target, 
                                                                test_size = 0.2, random_state = 11111)
f_train, f_valid, t_train, t_valid = train_test_split(f_train_valid, t_train_valid, 
                                                    test_size = 0.25, random_state = 11111)

print(f_train.shape)
print(f_valid.shape)
print(f_test.shape)

(95742, 153504)
(31914, 153504)
(31915, 153504)


Готово. Переходим к подбору модели.

# 2. Обучение и валидация

Помня, что перед нами задача классификации, воспользуемся логистической регрессией и градиентными глупыми деревьями от Майкрософт (LGBM).

Сперва - лог.рег.

In [23]:
model_logreg = LogisticRegression(random_state = 11111)

Повлиять на качество модели мы можем с помощью балансировки классов. Но не с помощью up- и down-грейда датасета, а с помощью балансировки классов. 
>class_weight = 'balanced'

In [24]:
%%time
model_logreg.fit(f_train, t_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 4.48 s, sys: 3.88 s, total: 8.36 s
Wall time: 8.36 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=11111, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

8 секунд на обучение модели! Неплохо, очень неплохо!

In [25]:
logreg_predict = model_logreg.predict(f_valid)
f1_score_logreg = f1_score(t_valid, logreg_predict)
f1_score_logreg

0.7020593236349896

Минимально допустимое значение метрики - 0.75.

Полученных 0.702 мало. Балансируем классы, переучиваем модель и смотрим на результат ещё раз.

In [26]:
model_logreg_balanced = LogisticRegression(random_state = 11111, class_weight = 'balanced')
model_logreg_balanced.fit(f_train, t_train)
logreg_balanced_predict = model_logreg_balanced.predict(f_valid)
f1_score_logreg_balanced = f1_score(t_valid, logreg_balanced_predict)
f1_score_logreg_balanced

0.7523887523887524

Парам-парам-пам! Пам! Фьюф!

Готово. Мы выше 0.75.

В дополнение ко всему, мы помним про возможность нахождения гиперпараметров. Даже у логистической регрессии. Запустим гридсёрч (это мой первый раз) и посмотрим на "лучший" вариант.

In [27]:
# это мы создаем "матрицу" варьирования значений регуляризации. 
# От 10 в -1 (0.1) до 10 в 1 (10). 3 значения на логарифмической шкале в этом диапазоне

c_space = np.logspace(-1, 1, 3)
param_grid = {'C': c_space}

model_logreg_gv = LogisticRegression(random_state = 11111)

logreg_cv = GridSearchCV(model_logreg_gv, param_grid, cv = 4, scoring = 'f1')

Воспользуемся специально заготовленным для этого случая датасета обучение+валидация.

In [28]:
%%time
logreg_cv.fit(f_train_valid, t_train_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 1min 5s, sys: 1min 5s, total: 2min 11s
Wall time: 2min 11s


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=11111, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([ 0.1,  1. , 10. ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [29]:
print(logreg_cv.best_params_)
print(logreg_cv.best_score_)

{'C': 10.0}
0.7612796752075379


В сравнении со стоковой моделью прирост почти в 0.06 для f1 - это весьма существено. 
Посмотрим, что можно получить для модели с балансировкой и регуляризацией. 

In [30]:
%%time
model_logreg_balanced_gv = LogisticRegression(random_state = 11111, class_weight = 'balanced')
logreg_balanced_cv = GridSearchCV(model_logreg_balanced_gv, param_grid, cv = 4, scoring = 'f1')
logreg_balanced_cv.fit(f_train_valid, t_train_valid)
print(logreg_balanced_cv.best_params_)
print(logreg_balanced_cv.best_score_)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'C': 10.0}
0.7596579690404472
CPU times: user 1min 40s, sys: 1min 47s, total: 3min 28s
Wall time: 3min 28s


Прироста к аналогичной модели без балансировки почти нет. 

Занесем полученные значения метрики и гиперпараметров в финальную таблицу и движемся дальше.

In [31]:
final_table = pd.DataFrame(data = [], columns = ['model','f1', 'c'])
final_table.loc[0, 'model'] = 'Логистическая регрессия'
final_table.loc[0, 'f1'] = f1_score_logreg

final_table.loc[1, 'model'] = 'Логистическая регрессия с балансировкой'
final_table.loc[1, 'f1'] = f1_score_logreg_balanced

final_table.loc[2, 'model'] = 'Логистическая регрессия с регуляризацией'
final_table.loc[2, 'f1'] = logreg_cv.best_score_
final_table.loc[2, 'c'] = logreg_cv.best_params_['C']

final_table.loc[3, 'model'] = 'Логистическая регрессия с балансировкой и регуляризацией'
final_table.loc[3, 'f1'] = logreg_balanced_cv.best_score_
final_table.loc[3, 'c'] = logreg_balanced_cv.best_params_['C']

final_table = final_table.fillna('-')
final_table

,model,f1,c
0,Логистическая регрессия,0.702059,-
1,Логистическая регрессия с балансировкой,0.752389,-
2,Логистическая регрессия с регуляризацией,0.761280,10
3,Логистическая регрессия с балансировкой и регу...,0.759658,10


К градиентному бустингу.

Разберемся с тем, какие гиперпараметры (и не только) нам следует взять для данной модели, чтобы она могла делать бинарную классификацию.

In [32]:
train_data = lgb.Dataset(f_train, label = t_train, free_raw_data = False)
valid_data = lgb.Dataset(f_valid, label = t_valid, free_raw_data = False)

In [33]:
parameters_gbdt = {'task' : 'train',
                   'boosting_type' : 'gbdt',
                   'objective' : 'binary',
                   'metric' : ['binary_logloss'],
                   'is_training_metric' : True,
                   'max_bin' : 255,
                   'learning_rate' : 0.05,
                   'num_leaves' : 100
                  }

In [34]:
best_model = lgb.train(parameters_gbdt, train_data, 20, valid_sets = valid_data, verbose_eval = 5)
    
pred_valid = best_model.predict(f_valid, num_iteration=best_model.best_iteration)

pred_valid_real = pred_valid.round()
f1_lgbm = f1_score(t_valid, pred_valid_real)
f1_lgbm

[5]	valid_0's binary_logloss: 0.252872
[10]	valid_0's binary_logloss: 0.220056
[15]	valid_0's binary_logloss: 0.201091
[20]	valid_0's binary_logloss: 0.188215


0.6367444643925793

Метрики f1 в данной библиотеке не обнаружено, поэтому вычисляем её самостоятельно. Предсказания модели - не готовые классы, а вероятность класса "1" - того, что комментарий токсичен. Поэтому, округляем значения вероятностей до целого числа (верояность 0.5 и ниже - 0 класс, всё оставльное - 1 класс).

> здесь кернел начал очень часто помирать. Не знаю, что сделать с этим. 

Посмотрим, чего можно добиться, увеличив число итераций.

In [35]:
best_model_100 = lgb.train(parameters_gbdt, train_data, 100, valid_sets = valid_data, verbose_eval = 5)

[5]	valid_0's binary_logloss: 0.252872
[10]	valid_0's binary_logloss: 0.220056
[15]	valid_0's binary_logloss: 0.201091
[20]	valid_0's binary_logloss: 0.188215
[25]	valid_0's binary_logloss: 0.179254
[30]	valid_0's binary_logloss: 0.171736
[35]	valid_0's binary_logloss: 0.165547
[40]	valid_0's binary_logloss: 0.159822
[45]	valid_0's binary_logloss: 0.155136
[50]	valid_0's binary_logloss: 0.150964
[55]	valid_0's binary_logloss: 0.147632
[60]	valid_0's binary_logloss: 0.144619
[65]	valid_0's binary_logloss: 0.141988
[70]	valid_0's binary_logloss: 0.139682
[75]	valid_0's binary_logloss: 0.137507
[80]	valid_0's binary_logloss: 0.13574
[85]	valid_0's binary_logloss: 0.134283
[90]	valid_0's binary_logloss: 0.132887
[95]	valid_0's binary_logloss: 0.131736
[100]	valid_0's binary_logloss: 0.130746


0.6367444643925793

In [37]:
pred_valid_100 = best_model_100.predict(f_valid, num_iteration=best_model.best_iteration)
pred_valid_real_100 = pred_valid_100.round()
f1_lgbm_100 = f1_score(t_valid, pred_valid_real_100)
f1_lgbm_100

0.7475386779184247

Больше итераций - существенно больше и показатель f1-меры. Да, время обучения также возрастает существенно. Зафиксируем полученные данные в таблице и перейдем к тестированию полученных моделей. 

In [43]:
final_table.loc[4, 'model'] = 'Градиентный бустинг, 20 итераций'
final_table.loc[4, 'f1'] = f1_lgbm
final_table.loc[4, 'num_iterations'] = 20

final_table.loc[5, 'model'] = 'Градиентный бустинг, 100 итераций'
final_table.loc[5, 'f1'] = f1_lgbm_100
final_table.loc[5, 'num_iterations'] = 100

final_table = final_table.fillna('-')
final_table

,model,f1,c,num_iterations
0,Логистическая регрессия,0.702059,-,-
1,Логистическая регрессия с балансировкой,0.752389,-,-
2,Логистическая регрессия с регуляризацией,0.761280,10,-
3,Логистическая регрессия с балансировкой и регу...,0.759658,10,-
4,"Градиентный бустинг, 20 итераций",0.636744,-,20
5,"Градиентный бустинг, 100 итераций",0.747539,-,100


# 3. Тестирование и выводы

### 1. Логистическая регрессия

In [51]:
f1_test_table = pd.DataFrame(data = [], columns = ['model', 'f1_score_test'])

pred_test_logreg = model_logreg.predict(f_test)
f1_test_table.loc[0, 'model'] = final_table.loc[0, 'model']
f1_test_table.loc[0, 'f1_score_test'] = f1_score(t_test, pred_test_logreg)
f1_test_table

,model,f1_score_test
0,Логистическая регрессия,0.707102


In [52]:
pred_test_logreg_balanced = model_logreg_balanced.predict(f_test)

f1_test_table.loc[1, 'model'] = final_table.loc[1, 'model']
f1_test_table.loc[1, 'f1_score_test'] = f1_score(t_test, pred_test_logreg_balanced)
f1_test_table

,model,f1_score_test
0,Логистическая регрессия,0.707102
1,Логистическая регрессия с балансировкой,0.742416


In [55]:
model_logreg_gv = LogisticRegression(C = 10, random_state = 11111)
model_logreg_gv.fit(f_train_valid, t_train_valid)

pred_test_logreg_gv = model_logreg_gv.predict(f_test)

f1_test_table.loc[2, 'model'] = final_table.loc[2, 'model']
f1_test_table.loc[2, 'f1_score_test'] = f1_score(t_test, pred_test_logreg_gv)
f1_test_table

,model,f1_score_test
0,Логистическая регрессия,0.707102
1,Логистическая регрессия с балансировкой,0.742416
2,Логистическая регрессия с регуляризацией,0.775307


In [56]:
model_logreg_gv_balanced = LogisticRegression(C = 10, class_weight = 'balanced', random_state = 11111)
model_logreg_gv_balanced.fit(f_train_valid, t_train_valid)

pred_test_logreg_gv_balanced = model_logreg_gv_balanced.predict(f_test)

f1_test_table.loc[3, 'model'] = final_table.loc[3, 'model']
f1_test_table.loc[3, 'f1_score_test'] = f1_score(t_test, pred_test_logreg_gv_balanced)
f1_test_table

,model,f1_score_test
0,Логистическая регрессия,0.707102
1,Логистическая регрессия с балансировкой,0.742416
2,Логистическая регрессия с регуляризацией,0.775307
3,Логистическая регрессия с балансировкой и регу...,0.763658


### 2. Градиентный бустинг

In [58]:
pred_gbdt_test = best_model.predict(f_test, num_iteration=best_model.best_iteration)
f1_test_table.loc[4, 'model'] = final_table.loc[4, 'model']
f1_test_table.loc[4, 'f1_score_test'] = f1_score(t_test, pred_gbdt_test.round())
f1_test_table

,model,f1_score_test
0,Логистическая регрессия,0.707102
1,Логистическая регрессия с балансировкой,0.742416
2,Логистическая регрессия с регуляризацией,0.775307
3,Логистическая регрессия с балансировкой и регу...,0.763658
4,"Градиентный бустинг, 20 итераций",0.637729


In [59]:
pred_gbdt_100_test = best_model_100.predict(f_test, num_iteration=best_model_100.best_iteration)
f1_test_table.loc[5, 'model'] = final_table.loc[5, 'model']
f1_test_table.loc[5, 'f1_score_test'] = f1_score(t_test, pred_gbdt_100_test.round())
f1_test_table

,model,f1_score_test
0,Логистическая регрессия,0.707102
1,Логистическая регрессия с балансировкой,0.742416
2,Логистическая регрессия с регуляризацией,0.775307
3,Логистическая регрессия с балансировкой и регу...,0.763658
4,"Градиентный бустинг, 20 итераций",0.637729
5,"Градиентный бустинг, 100 итераций",0.750136


### Выводы

Удивительно, но факт. По итогу эксперимента, пороговое значение прошло сразу 3 модели из 6 - Их вы можете наблюдать в таблице ниже. Причем, что важно, в ряде случаев, точность модели даже увеличивалась по сравнению с значениями для валидационной выборки. 

В нашем случае логистическая регрессия c регуляризацией (С = 10) - топ 1 по значению f1 - метрики взвешенной точности и полноты.

In [62]:
f1_test_table['f1_valid'] = final_table['f1']
f1_test_table.sort_values(by = 'f1_score_test', ascending = False)

,model,f1_score_test,f1_valid
2,Логистическая регрессия с регуляризацией,0.775307,0.761280
3,Логистическая регрессия с балансировкой и регу...,0.763658,0.759658
5,"Градиентный бустинг, 100 итераций",0.750136,0.747539
1,Логистическая регрессия с балансировкой,0.742416,0.752389
0,Логистическая регрессия,0.707102,0.702059
4,"Градиентный бустинг, 20 итераций",0.637729,0.636744


Да, в рамках проекта можно было получить ЛУЧШИЕ результаты - я точно это знаю и уверен, что приложив чуть больше усилий, я бы мог добиться большей точности за счёт:

1) более качественной предобработки данных и извлечения признаков:
- убрав одиночные буквы
- воспользовавшишься иным способом создания признаков
- оставив апострофы в нужных местах
- воспользовавшись БЕРТом

2) использования других моделей машинного обучения и настройки гиперпараметров существующих моделей.
- в случае логистической регрессии стоило пройти более широкий диапазон значений C - с меньшим шагом. Это позволило бы ещё повысить значение f1, не "сорвавшись" при этом в "пропасть переобучения".
- в случае градиентного бустинга - пройтись гридСёрчем по значениям скорости обучения, числа итераций, числа листьев, размера батча и т.д. И 100%, мы бы достигли лучшего и по времени и по точности значения f1-меры. 

Однако, в моей жизни происходит куча сложных моментов, которые мне тяжело вывозить психологически и у меня просто нет сил дальше "прорабатывать" этот проект.

Спасибо вам за вашу работу!